In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer
import nltk
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from joblib import dump

In [ ]:
#Loading the dataset
data = pd.read_excel('hotel_reviews.xlsx') 

data.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4.0
1,ok nothing special charge diamond member hilto...,2.0
2,nice rooms not 4* experience hotel monaco seat...,3.0
3,"unique, great stay, wonderful time hotel monac...",5.0
4,"great stay great stay, went seahawk game aweso...",5.0


In [ ]:
# Creating a new column sentiment based on overall ratings
def compute_sentiments(labels):
  sentiments = []
  for label in labels:
    if label > 3.0:
      sentiment = 1
    elif label <= 3.0:
      sentiment = 0
    sentiments.append(sentiment)
  return sentiments

In [ ]:
data['Rating'] = compute_sentiments(data.Rating)

In [ ]:
data.columns = ['Review', 'Sentiment']

data.head()

,Review,Sentiment
0,nice hotel expensive parking got good deal sta...,1
1,ok nothing special charge diamond member hilto...,0
2,nice rooms not 4* experience hotel monaco seat...,0
3,"unique, great stay, wonderful time hotel monac...",1
4,"great stay great stay, went seahawk game aweso...",1


In [ ]:
# check distribution of sentiments

data['Sentiment'].value_counts()

1    15093
0     5398
Name: Sentiment, dtype: int64

In [ ]:
# check for null values
data.isnull().sum()

# no null values in the data

Review       0
Sentiment    0
dtype: int64

### Data Cleaning

In [ ]:
text = data["Review"][1]
text

'ok nothing special charge diamond member hilton decided chain shot 20th anniversary seattle, start booked suite paid extra website description not, suite bedroom bathroom standard hotel room, took printed reservation desk showed said things like tv couch ect desk clerk told oh mixed suites description kimpton website sorry free breakfast, got kidding, embassy suits sitting room bathroom bedroom unlike kimpton calls suite, 5 day stay offer correct false advertising, send kimpton preferred guest website email asking failure provide suite advertised website reservation description furnished hard copy reservation printout website desk manager duty did not reply solution, send email trip guest survey did not follow email mail, guess tell concerned guest.the staff ranged indifferent not helpful, asked desk good breakfast spots neighborhood hood told no hotels, gee best breakfast spots seattle 1/2 block away convenient hotel does not know exist, arrived late night 11 pm inside run bellman bu

In [ ]:
cleantext = BeautifulSoup(text, "lxml").text
cleantext

'ok nothing special charge diamond member hilton decided chain shot 20th anniversary seattle, start booked suite paid extra website description not, suite bedroom bathroom standard hotel room, took printed reservation desk showed said things like tv couch ect desk clerk told oh mixed suites description kimpton website sorry free breakfast, got kidding, embassy suits sitting room bathroom bedroom unlike kimpton calls suite, 5 day stay offer correct false advertising, send kimpton preferred guest website email asking failure provide suite advertised website reservation description furnished hard copy reservation printout website desk manager duty did not reply solution, send email trip guest survey did not follow email mail, guess tell concerned guest.the staff ranged indifferent not helpful, asked desk good breakfast spots neighborhood hood told no hotels, gee best breakfast spots seattle 1/2 block away convenient hotel does not know exist, arrived late night 11 pm inside run bellman bu

In [ ]:
#https://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string
cleantext = re.sub(r'[^\w\s]','',cleantext)
cleantext

'ok nothing special charge diamond member hilton decided chain shot 20th anniversary seattle start booked suite paid extra website description not suite bedroom bathroom standard hotel room took printed reservation desk showed said things like tv couch ect desk clerk told oh mixed suites description kimpton website sorry free breakfast got kidding embassy suits sitting room bathroom bedroom unlike kimpton calls suite 5 day stay offer correct false advertising send kimpton preferred guest website email asking failure provide suite advertised website reservation description furnished hard copy reservation printout website desk manager duty did not reply solution send email trip guest survey did not follow email mail guess tell concerned guestthe staff ranged indifferent not helpful asked desk good breakfast spots neighborhood hood told no hotels gee best breakfast spots seattle 12 block away convenient hotel does not know exist arrived late night 11 pm inside run bellman busy chating cel

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
cleantext= cleantext.lower()
stopword = set(stopwords.words('english'))
tokens= cleantext.split()
token_list = []
for token in tokens:
    if token not in stopword:
        token_list.append(token)
     


In [ ]:
stopword = set(stopwords.words('english'))

In [ ]:

token_list

['ok',
 'nothing',
 'special',
 'charge',
 'diamond',
 'member',
 'hilton',
 'decided',
 'chain',
 'shot',
 '20th',
 'anniversary',
 'seattle',
 'start',
 'booked',
 'suite',
 'paid',
 'extra',
 'website',
 'description',
 'suite',
 'bedroom',
 'bathroom',
 'standard',
 'hotel',
 'room',
 'took',
 'printed',
 'reservation',
 'desk',
 'showed',
 'said',
 'things',
 'like',
 'tv',
 'couch',
 'ect',
 'desk',
 'clerk',
 'told',
 'oh',
 'mixed',
 'suites',
 'description',
 'kimpton',
 'website',
 'sorry',
 'free',
 'breakfast',
 'got',
 'kidding',
 'embassy',
 'suits',
 'sitting',
 'room',
 'bathroom',
 'bedroom',
 'unlike',
 'kimpton',
 'calls',
 'suite',
 '5',
 'day',
 'stay',
 'offer',
 'correct',
 'false',
 'advertising',
 'send',
 'kimpton',
 'preferred',
 'guest',
 'website',
 'email',
 'asking',
 'failure',
 'provide',
 'suite',
 'advertised',
 'website',
 'reservation',
 'description',
 'furnished',
 'hard',
 'copy',
 'reservation',
 'printout',
 'website',
 'desk',
 'manager',
 'du

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
lemmatizer.lemmatize("hotels")

'hotel'

In [ ]:
[lemmatizer.lemmatize(token) for token in token_list]

['ok',
 'nothing',
 'special',
 'charge',
 'diamond',
 'member',
 'hilton',
 'decided',
 'chain',
 'shot',
 '20th',
 'anniversary',
 'seattle',
 'start',
 'booked',
 'suite',
 'paid',
 'extra',
 'website',
 'description',
 'suite',
 'bedroom',
 'bathroom',
 'standard',
 'hotel',
 'room',
 'took',
 'printed',
 'reservation',
 'desk',
 'showed',
 'said',
 'thing',
 'like',
 'tv',
 'couch',
 'ect',
 'desk',
 'clerk',
 'told',
 'oh',
 'mixed',
 'suite',
 'description',
 'kimpton',
 'website',
 'sorry',
 'free',
 'breakfast',
 'got',
 'kidding',
 'embassy',
 'suit',
 'sitting',
 'room',
 'bathroom',
 'bedroom',
 'unlike',
 'kimpton',
 'call',
 'suite',
 '5',
 'day',
 'stay',
 'offer',
 'correct',
 'false',
 'advertising',
 'send',
 'kimpton',
 'preferred',
 'guest',
 'website',
 'email',
 'asking',
 'failure',
 'provide',
 'suite',
 'advertised',
 'website',
 'reservation',
 'description',
 'furnished',
 'hard',
 'copy',
 'reservation',
 'printout',
 'website',
 'desk',
 'manager',
 'duty',

In [ ]:
lemma_word = []
for token in token_list:
    lemma_word.append(lemmatizer.lemmatize(token))
lemma_word

['ok',
 'nothing',
 'special',
 'charge',
 'diamond',
 'member',
 'hilton',
 'decided',
 'chain',
 'shot',
 '20th',
 'anniversary',
 'seattle',
 'start',
 'booked',
 'suite',
 'paid',
 'extra',
 'website',
 'description',
 'suite',
 'bedroom',
 'bathroom',
 'standard',
 'hotel',
 'room',
 'took',
 'printed',
 'reservation',
 'desk',
 'showed',
 'said',
 'thing',
 'like',
 'tv',
 'couch',
 'ect',
 'desk',
 'clerk',
 'told',
 'oh',
 'mixed',
 'suite',
 'description',
 'kimpton',
 'website',
 'sorry',
 'free',
 'breakfast',
 'got',
 'kidding',
 'embassy',
 'suit',
 'sitting',
 'room',
 'bathroom',
 'bedroom',
 'unlike',
 'kimpton',
 'call',
 'suite',
 '5',
 'day',
 'stay',
 'offer',
 'correct',
 'false',
 'advertising',
 'send',
 'kimpton',
 'preferred',
 'guest',
 'website',
 'email',
 'asking',
 'failure',
 'provide',
 'suite',
 'advertised',
 'website',
 'reservation',
 'description',
 'furnished',
 'hard',
 'copy',
 'reservation',
 'printout',
 'website',
 'desk',
 'manager',
 'duty',

In [ ]:
join_text = ' '.join(lemma_word)
join_text

'ok nothing special charge diamond member hilton decided chain shot 20th anniversary seattle start booked suite paid extra website description suite bedroom bathroom standard hotel room took printed reservation desk showed said thing like tv couch ect desk clerk told oh mixed suite description kimpton website sorry free breakfast got kidding embassy suit sitting room bathroom bedroom unlike kimpton call suite 5 day stay offer correct false advertising send kimpton preferred guest website email asking failure provide suite advertised website reservation description furnished hard copy reservation printout website desk manager duty reply solution send email trip guest survey follow email mail guess tell concerned guestthe staff ranged indifferent helpful asked desk good breakfast spot neighborhood hood told hotel gee best breakfast spot seattle 12 block away convenient hotel know exist arrived late night 11 pm inside run bellman busy chating cell phone help bagsprior arrival emailed hote

In [ ]:
def data_cleaner(data):
    clean_data = []
    for sentence in tqdm(data):
        cleantext = BeautifulSoup(sentence, "lxml").text #html tags
        cleantext = re.sub(r'[^\w\s]','',cleantext) # punctuation
        cleantext = [token for token in cleantext.lower().split() if token not in stopword] #stopword
        clean_text = ' '.join([lemmatizer.lemmatize(token) for token in cleantext])
        clean_data.append(clean_text.strip())
    return clean_data

In [ ]:
clean_data = data_cleaner(data.Review.values)

100%|██████████| 20491/20491 [00:11<00:00, 1764.84it/s]


In [ ]:
clean_data[100]

'great location staff location price make older hotel good choice staff friendly looking modern place old studio apartment building run hotel great neighborhood close great local resturants hotel sterile tourist feeling newer mid range hotel feel like small european hotel yes room use little moderizing compermise reservation stay'

### Vectorization Feature Engineering (TF-IDF)

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [ ]:
vec = TfidfVectorizer()
vec.fit(clean_data)

TfidfVectorizer()

## Train the model

### Train/ Test Split

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = data.Sentiment, random_state = 0)

NameError: ignored

In [ ]:
x_train.shape, x_test.shape
# 2520 samples in training dataset and 630 in test dataset

### Fit x_train and y_train

In [ ]:
classifier = LinearSVC()

In [ ]:
pipeline = Pipeline([('tfidf',vec), ('clf',classifier)])
# it will first do vectorization and then it will do classification

In [ ]:
pipeline.fit(x_train, y_train)

In [ ]:
# in this we don't need to prepare the dataset for testing(x_test)

In [ ]:
import joblib
joblib.dump(pipeline,'sentiment_model.pkl')

## Check Model Performance

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
y_pred = pipeline.predict(x_test)

In [ ]:
# confusion_matrix
confusion_matrix(y_test, y_pred)

array([[ 821,  259],
       [ 181, 2838]])

In [ ]:
# classification_report
print(classification_report(y_test, y_pred))
# we are getting almost 91% accuracy

              precision    recall  f1-score   support

           0       0.82      0.76      0.79      1080
           1       0.92      0.94      0.93      3019

    accuracy                           0.89      4099
   macro avg       0.87      0.85      0.86      4099
weighted avg       0.89      0.89      0.89      4099



In [ ]:
round(accuracy_score(y_test, y_pred)*100,2)

89.27

## Predict Sentiments using Model

In [ ]:
 prediction = pipeline.predict(["hotel is bad"])

 if prediction == 1:
   print("Result: This review is positive")
 else:
   print("Result: This review is negative")

Result: This review is negative
